In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [43]:
diabetes = pd.read_csv('pima_indians_diabetes.csv')
sonar = pd.read_csv('sonar.csv')
ionosphere = pd.read_csv('ionosphere.csv')
glass = pd.read_csv('glass.csv')
data_dict = {'diabetes':(diabetes,(10,5,),30),
             'sonar': (sonar,(20,10,), 60),
             'glass': (glass, (20,10,), 80),
             'ionosphere': (ionosphere,(20,10,),40)}

In [44]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, KFold

In [45]:
class CustomMLP(MLPClassifier):
    def _init_coef(self, fan_in, fan_out):
        weights = np.random.uniform(-0.5,0.5,(fan_in,fan_out))
        bias = np.ones((fan_out,))
        return weights, bias

In [46]:
all_errors = {}
for key, values in data_dict.items():
    data, hidden, epochs = values
    classifier = CustomMLP(hidden_layer_sizes=hidden,
                           learning_rate_init=0.15,momentum=0.9, nesterovs_momentum=False,
                           max_iter=epochs, batch_size=data.shape[0], shuffle=False,
                           activation='logistic',solver='sgd',alpha=0.0,
                           n_iter_no_change=epochs)
    errors = []
    for i in range(5):
        errors.append(1-cross_val_score(classifier,data.iloc[:,:-1],data.iloc[:,-1],cv=10))
    all_errors[key] = errors

In [42]:
[print(f'{key}: {np.mean(x)*100}') for key,x in all_errors.items()]

diabetes: 34.99897470950103
sonar: 39.71904761904762
glass: 33.78787878787878
ionosphere: 15.446031746031744


[None, None, None, None]

In [47]:
[print(f'{key}: {np.mean(x)*100}') for key,x in all_errors.items()]

diabetes: 34.89405331510594
sonar: 47.0047619047619
glass: 31.16017316017316
ionosphere: 35.32539682539682


[None, None, None, None]

In [29]:
for key, val in all_errors.items():
    print(f'{key}: {[x.mean() for x in val]}')


diabetes: [0.3489917976760082, 0.3515721120984279, 0.34637730690362273, 0.35553656869446343, 0.3515721120984279]
sonar: [0.6478571428571429, 0.6285714285714286, 0.6283333333333334, 0.6814285714285715, 0.6047619047619047]
glass: [0.5318181818181819, 0.36753246753246754, 0.3829004329004329, 0.3926406926406926, 0.4374458874458874]
ionosphere: [0.1565079365079365, 0.1907936507936508, 0.19087301587301586, 0.1537301587301587, 0.1938095238095238]


In [31]:
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier

In [37]:
import warnings
warnings.filterwarnings('ignore')
all_errors = {}
for key, values in data_dict.items():
    data, hidden, epochs = values
    classifier = CustomMLP(hidden_layer_sizes=hidden,
                           learning_rate_init=0.15,momentum=0.9, nesterovs_momentum=False,
                           max_iter=epochs, batch_size=data.shape[0], shuffle=False,
                           activation='logistic',solver='sgd',alpha=0.0,
                           n_iter_no_change=epochs)
    bagger = BaggingClassifier(classifier, n_estimators=25)
    booster = AdaBoostClassifier(n_estimators=25)
    errors = {'Stan':[], 'Bag':[], 'Ada':[]}
    for i in range(5):
#         errors['Stan'].append(1-cross_val_score(classifier,data.iloc[:,:-1],data.iloc[:,-1],cv=10))
#         errors['Bag'].append(1-cross_val_score(bagger,data.iloc[:,:-1],data.iloc[:,-1],cv=10))
        errors['Ada'].append(1-cross_val_score(booster,data.iloc[:,:-1],data.iloc[:,-1],cv=10))
    all_errors[key] = errors
#     print(f"{key}: {np.mean(errors['Stan'])}, {np.mean(errors['Bag'])}, {np.mean(errors['Ada'])}")
    print(f"{key}: {errors['Ada']}")

diabetes: [array([0.2987013 , 0.19480519, 0.27272727, 0.33766234, 0.28571429,
       0.23376623, 0.18181818, 0.20779221, 0.26315789, 0.15789474]), array([0.2987013 , 0.19480519, 0.27272727, 0.33766234, 0.28571429,
       0.23376623, 0.18181818, 0.20779221, 0.26315789, 0.15789474]), array([0.2987013 , 0.19480519, 0.27272727, 0.33766234, 0.28571429,
       0.23376623, 0.18181818, 0.20779221, 0.26315789, 0.15789474]), array([0.2987013 , 0.19480519, 0.27272727, 0.33766234, 0.28571429,
       0.23376623, 0.18181818, 0.20779221, 0.26315789, 0.15789474]), array([0.2987013 , 0.19480519, 0.27272727, 0.33766234, 0.28571429,
       0.23376623, 0.18181818, 0.20779221, 0.26315789, 0.15789474])]
sonar: [array([0.57142857, 0.19047619, 0.14285714, 0.19047619, 0.14285714,
       0.23809524, 0.        , 0.33333333, 0.4       , 0.35      ]), array([0.57142857, 0.19047619, 0.14285714, 0.19047619, 0.14285714,
       0.23809524, 0.        , 0.33333333, 0.4       , 0.35      ]), array([0.57142857, 0.19047619